# Question 2: Operations Excellence of plants

You operate two plants, i.e., A and B. Each plant makes two products, “standard” and “deluxe”. A unit of
standard gives a profit contribution of \\$10, while a unit of deluxe gives a profit contribution of $15.

Each plant uses two processes, grinding and polishing, for producing its products. Plant A has a grinding capacity of 80 hours per week and polishing capacity of 60 hours per week. For plant B, these capacities are 60 and 75 hours per week, respectively.
The grinding and polishing times in hours for a unit of each type of product in each factory are as given below.

For Plant A
Standard Deluxe
Grinding 4 2
Polishing 2 5
For Plant B
Standard Deluxe
Grinding 5 3
Polishing 5 6

It is possible, for example, that plant B has older machines than plant A, resulting in higher unit processing times. In addition, each unit of each product uses 4 kg of a raw material, which we refer to as raw. The company has 120 kg of raw available per week. To start with, we will assume that plant A is allocated 75 kg of raw per week and plant B the remaining 45 kg per week. Each plant can build a very simple linear programming model to maximize its profit contribution.

### Import Libraries

In [0]:
import math
import numpy
import cvxpy as cvx
import numpy
from prettytable import PrettyTable

### Variable declaration

In [0]:
# number of products produced by each plant
a1 = cvx.Variable(1, integer=True)# Standard from Plant A
a2 = cvx.Variable(1, integer=True)# Deluxe from Plant A
b1 = cvx.Variable(1, integer=True)# Standard from Plant B
b2 = cvx.Variable(1, integer=True)# Deluxe from Plant B

### Constraints

In [0]:
c1= a1 >= 0
c2= a2 >= 0 
c3= b1 >= 0 
c4= b2 >= 0 
c5= a1*4 + a2*2 <= 80
c6= a1*2 + a2*5 <= 60
c7= b1*5 + b2*3 <= 60
c8= b1*5 + b2*6 <= 75
c9= a1*4 + a2*4 <= 75
c10= b1*4 + b2*4 <= 45

con_A = [c1,c2,c5,c6,c9]
con_B = [c3,c4,c7,c8,c10]
con = [c1,c2,c3,c4,c5,c6,c7,c8,c9,c10]

### Objective

In [0]:
#objective

std_price = 10 # price of standard product
dlx_price = 15 # price of deluxe product

OF_A = a1*std_price + a2*dlx_price
OF_B = b1*std_price + b2*dlx_price
OF = a1*std_price + a2*dlx_price + b1*std_price + b2*dlx_price

objective_A = cvx.Maximize(OF_A)
objective_B = cvx.Maximize(OF_B)
objective = cvx.Maximize(OF)



### Plant A: Maximum Revenue & Allocation

In [0]:
#solving - Plant A
prob_A = cvx.Problem(objective_A, con_A)
result_A = prob_A.solve()
print('\nPLANT A')

print('optimal revenue')
print (int(np.round(prob_A.value)))
    
xa = PrettyTable()

xa.field_names = ["Product", "Plant A"]

xa.add_row(["Standard", int(np.round(a1.value))])
xa.add_row(["Deluxe", int(np.round(a2.value))])

print(xa)




PLANT A
optimal revenue
220
+----------+---------+
| Product  | Plant A |
+----------+---------+
| Standard |    10   |
|  Deluxe  |    8    |
+----------+---------+


To attain a maximum profit in Plant A, it should produce ten Standard products and eight Deluxe products. This would yield in a revenue of $220.

### Plant B: Maximum Revenue & Allocation

In [0]:
#solving - Plant B
prob_B = cvx.Problem(objective_B, con_B)
result_B = prob_B.solve()

print('\nPLANT B')
print('optimal revenue')
print (int(np.round(prob_B.value)))

xb = PrettyTable()

xb.field_names = ["Product", "Plant B"]

xb.add_row(["Standard", int(np.round(b1.value))])
xb.add_row(["Deluxe", int(np.round(b2.value))])
print(xb)


PLANT B
optimal revenue
165
+----------+---------+
| Product  | Plant B |
+----------+---------+
| Standard |    0    |
|  Deluxe  |    11   |
+----------+---------+


To attain a maximum profit in Plant B, it should produce eleven Deluxe products. This would yield in a revenue of $165.

### Analysis I: Plant A and Plant B: Maximum Revenue & Allocation

In [0]:
#solving - Plant A and B
prob = cvx.Problem(objective, con)
result = prob.solve()
print('\nPLANT A & B')
print('optimal revenue')
print (int(np.round(prob.value)))

x = PrettyTable()

x.field_names = ["Product", "Plant A", "Plant B"]

x.add_row(["Standard", int(np.round(a1.value)), int(np.round(b1.value))])
x.add_row(["Deluxe", int(np.round(a2.value)), int(np.round(b2.value))])
print(x)


PLANT A & B
optimal revenue
385
+----------+---------+---------+
| Product  | Plant A | Plant B |
+----------+---------+---------+
| Standard |    10   |    0    |
|  Deluxe  |    8    |    11   |
+----------+---------+---------+


Compared to the solution from individual plant profit contribution, the allocation are more or less same if they do not share raw material between them.


### Analysis II: Alternative Solution - sharing raw material

In [0]:
#solving - Plant A and B

# new constraint - sharing the raw material between A and B instead of allocating fixed amount to each plant
c11 = 4*(a1+a2+b1+b2) <= 120
con = [c1,c2,c3,c4,c5,c6,c7,c8,c11]
prob = cvx.Problem(objective, con)
result = prob.solve()
print('\nPLANT A & B')
print('optimal revenue')
print (int(np.round(prob.value)))

x = PrettyTable()

x.field_names = ["Product", "Plant A", "Plant B"]

x.add_row(["Standard", int(np.round(a1.value)), int(np.round(b1.value))])
x.add_row(["Deluxe", int(np.round(a2.value)), int(np.round(b2.value))])
print(x)



PLANT A & B
optimal revenue
400
+----------+---------+---------+
| Product  | Plant A | Plant B |
+----------+---------+---------+
| Standard |    10   |    0    |
|  Deluxe  |    8    |    12   |
+----------+---------+---------+


If they share raw material, their revenue increases from \\$385 to $400.



### Analysis III: Better Solution - Unlimited supply of raw material

In [0]:

#solving - Plant A and B
# removing the constraint on raw material
# if company is able to provide infinite amount of raw material to both plants
con = [c1,c2,c3,c4,c5,c6,c7,c8]
prob = cvx.Problem(objective, con)
result = prob.solve()
print('\nPLANT A & B')
print('optimal revenue')
print (int(np.round(prob.value)))

x = PrettyTable()

x.field_names = ["Product", "Plant A", "Plant B"]

x.add_row(["Standard", int(np.round(a1.value)), int(np.round(b1.value))])
x.add_row(["Deluxe", int(np.round(a2.value)), int(np.round(b2.value))])
print(x)


PLANT A & B
optimal revenue
425
+----------+---------+---------+
| Product  | Plant A | Plant B |
+----------+---------+---------+
| Standard |    17   |    0    |
|  Deluxe  |    5    |    12   |
+----------+---------+---------+


Based on the constraints we have received, we have created our optimization model and perofrmed optimization. We have done three types of analysis pertaining to different conditions and I would love to go over them one by one. Each of the analysis differes the way raw material is distributed among the two plants A and B.

In our first analysis, we have created our constraints based on "strict allocation." When Plant A and B are given 75 and 45 raw material each, we are able to see a profit of $385. Please refer to Analysis I for the allocation details.

In our second analysis, we have created our constraints based on "sharing resources." When Plant A and B are together given 120 amount of raw material we are able to see a profit of $400. Please refer to Analysis II for the allocation details. 

In our final analysis, we have created our constraints based on "unlimited supply." When Plant A and B are given unlimited supply of raw materials, we are able to see a profit of $425. Please refer to Analysis III for the allocation details.

We ran the above three analysis to find the optimal solution using our optimization model by changing our constraints. If you can purchase 136 quantities of raw materials and give 68 to each plant A and Plant B, they can make complete use of the resources allocated to them and be return maximum profit. 

Let us know how much more raw materials you could purchase, so that we can run our model with the new constraints and provide better results.